In [25]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pysal.explore import inequality
from inequality.gini import Gini_Spatial
from pysal.lib import weights
import warnings

In [2]:
path = "D:/Dumps/GEE-WP Scores/E2SFCA/100m grids/grid_geoms/"
cities_cont = 'Addis Ababa|Bologna|Cochabamba|Damascus|Detroit|Indore|Kampala|Memphis|Seattle|Shijiazhuang'
ls_file = os.listdir(path)
ls_file = pd.Series(ls_file)[pd.Series(ls_file).str.contains(cities_cont)].tolist()
city_score = [gpd.read_file(path+i) for i in ls_file]

In [ ]:
%%time
warnings.filterwarnings('ignore')
cities_gini = []
cities_spatialgini = []
for city in enumerate(city_score):
    tr_gini = []
    tr_spatialgini = []
    print(ls_file[city[0]].split('.')[0])
    for i in city[1].columns[city[1].columns.str.contains('Sc-access')]:
        float_df = city[1][['geometry','population',i]]

        gini = pd.DataFrame(float_df[i].value_counts()).sort_index()
        gini['pop'] = float_df.groupby(i)['population'].sum()
        gini = gini.reset_index()
        gini['frac_pop'] = gini['pop'] / gini['pop'].sum()
        gini['richer_frac'] = 1-gini['frac_pop'].cumsum()
        gini['ugs'] = (gini['index'] * gini['pop']) 
        gini['frac_ugs'] = gini.ugs/gini.ugs.sum()
        gini_score = 1-(gini.frac_ugs*(gini.frac_pop+2*gini.richer_frac)).sum()
        print('gini',i.rsplit(' ')[-1],round(gini_score,4))
        tr_gini.append(gini_score)
        
        wq = weights.Queen.from_dataframe(float_df)
        wq.transform = "B"
        spatial_gini = Gini_Spatial(float_df[i], wq).g
        print('spatial gini',i.rsplit(' ')[-1],round(spatial_gini,4))
        tr_spatialgini.append(spatial_gini)
    cities_gini.append(tr_gini)
    cities_spatialgini.append(tr_spatialgini)
    print(' ')

Addis Ababa
gini 300 0.9645
spatial gini 300 0.961
gini 600 0.9532
spatial gini 600 0.956
gini 1000 0.9357
spatial gini 1000 0.9479
 
Bologna
gini 300 0.899
spatial gini 300 0.9145
gini 600 0.8257
spatial gini 600 0.8688
gini 1000 0.7572
spatial gini 1000 0.8144
 
Cochabamba
gini 300 0.9599
spatial gini 300 0.9889
gini 600 0.9375
spatial gini 600 0.9839
gini 1000 0.9276
spatial gini 1000 0.9794
 
Damascus
gini 300 0.865
spatial gini 300 0.9032
gini 600 0.7816
spatial gini 600 0.8406
gini 1000 0.7046
spatial gini 1000 0.7819
 
Detroit
gini 300 0.9564
spatial gini 300 0.9789
gini 600 0.8992
spatial gini 600 0.9572
gini 1000 0.8692
spatial gini 1000 0.9415
 
Indore
gini 300 0.9592
spatial gini 300 0.9927
gini 600 0.9275
spatial gini 600 0.9864
gini 1000 0.9106
spatial gini 1000 0.9831
 
Kampala
gini 300 0.9464
spatial gini 300 0.9487
gini 600 0.8761
spatial gini 600 0.8881
gini 1000 0.7866
spatial gini 1000 0.8132
 
Memphis
gini 300 0.9909
spatial gini 300 0.9902
gini 600 0.9752
